In [1]:
#| code-fold: true

################################################################################

# autoreload all modules every time before executing the Python code
%reload_ext autoreload
%autoreload 2

################################################################################

from IPython.core.interactiveshell import InteractiveShell

# `ast_node_interactivity` is a setting that determines how the return value of the last line in a cell is displayed
# with `last_expr_or_assign`, the return value of the last expression is displayed unless it is assigned to a variable
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

################################################################################

import pandas as pd

# `copy_on_write` is a performance improvement
# This will be the default in a future version of pandas
# Refer to https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
pd.options.mode.copy_on_write = True
pd.options.future.no_silent_downcasting = True

################################################################################

%matplotlib inline

import matplotlib as mpl

mpl.use("agg")

# `constrained_layout` helps avoid overlapping elements
# Refer to https://matplotlib.org/stable/tutorials/intermediate/constrainedlayout_guide.html
mpl.pyplot.rcParams["figure.constrained_layout.use"] = True

import holoviews as hv
import hvplot.pandas  # noqa
import matplotlib as mpl
import pandas as pd
import panel as pn  # noqa
import param as pm

pn.extension("tabulator")
hv.extension("bokeh")

[`panel`](https://panel.holoviz.org/) is a library that allows creating interactive dashboards in pure Python. It's a flexible library that allows interlinking matplotlib, bokeh, widgets, and more.

There are quite a few ways to use `panel`, some of which are better for certain use cases. I wanted to write this post to share some tips and tricks that I have learned while using `panel`.

## Development Constraints

My first constraint when using `panel` was that I wanted to develop the dashboard in an interactive manner, preferably using a Jupyter notebook environment. `panel` does allow starting a server using the `panel serve` command and you can pass in the path to a file or a jupyter notebook. However, I only want to prototype individual components in a Jupyter notebook, and once I'm happy with the components, I want to combine them into a dashboard. For multi-page dashboards, I didn't want to have to store different components in different files and run multiple servers. 

My second constraint was that I wanted to make the dashboard as modular as possible. I wanted to be able to reuse some components across different dashboards. For example, I might have a component that shows a line chart and I want to use that line chart in a "user guide page" as well as in the "main dashboard".

Lastly, I wanted the code to be usable in a Python script in case someone wanted to programmatically access the same features. Imagine if a user wanted to plot the line chart from the dashboard and annotate it with some custom text. I wanted to make it easy for an advanced user that wanted to do that to have the option to do so.

The most natural way to do this was to make the components as classes and then instantiate them as part of the dashboard.
For this post, I'm going to use the IMDb movies dataset as an example to build a dashboard.

## Panel Components

`panel` is built on top of `param`. One useful way to think about `panel` and `param` is that `param` is a way to define state and `panel` is a way to visualize that state. And making state driven components is a great way to make interactive dashboards.

```{mermaid}
graph TD
    A[param: Define State] --> B[panel: Visualize State]

```

I like to start off by making a class that defines the state of a component. This usually involves understanding a few different things:

1. What are the inputs to the component?
2. What are the derived properties of the component?
3. What are the outputs of the component?



### Identifying Inputs

Any state of your application should be stored as a parameter. Let's say I want to filter based on the year, the average ratings and the runtime of movies. All of these should be stored as parameters to make a reactive application.  

In [11]:
class MoviesStateExample(pm.Parameterized):
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()
    year_range = pm.Range()
    ratings_range = pm.Range()
    runtime_range = pm.Range()

### Identifying Derived Properties

In this example, when the class is initialized, I want to load the CSV files, preprocess and clean the data.
When `self.df = df` is called, `param` will trigger an action with the name of the parameter. And any functions that are listening to that action will be called.

As far as I can tell, there are 3 ways to listen to changes in a parameter.

1. Add a member function with the `param.depends` decorator.

In [12]:
class MovieUsingDepends(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = self.df["startYear"].min()
        self.end_year = self.df["startYear"].max()


m = MovieUsingDepends()
print(m.start_year, m.end_year)

1892 1912


2. Use `pm.bind` with the `watch=True` argument.

In [13]:
class MovieUsingBind(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        pm.bind(self._update_bounds, self.param.df, watch=True)
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    def _update_bounds(self, df):
        self.start_year = df["startYear"].min()
        self.end_year = df["startYear"].max()


m = MovieUsingBind()
print(m.start_year, m.end_year)

1892 1912


3. Use `self.param.watch` in the `__init__` function.

In [14]:
class MovieUsingWatch(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.param.watch(self._update_bounds, "df")
        self.df = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)

    def _update_bounds(self, event):
        if event.name == "df":
            df = event.new
            self.start_year = df["startYear"].min()
            self.end_year = df["startYear"].max()


m = MovieUsingWatch()
print(m.start_year, m.end_year)

1892 1912



::: {.callout-note}

I personally prefer using `param.depends` because it is more explicit and easier to read.

:::

In all cases, when you use `watch=True`, you have created a dependent function, and any properties that are updated in that function are dependent properties. 

### Outputs

Finally, it is important to define the outputs of the component. In this case, the outputs are the visualization and the UI components.

In [15]:
class MovieWithOutputs(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv(
            "./data/title.basics.tsv.gz",
            sep="\t",
            usecols=["primaryTitle", "startYear"],
            nrows=200,
        )

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = self.df["startYear"].min()
        self.end_year = self.df["startYear"].max()

    @pm.depends("df", "start_year", "end_year")
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        return df.query(f"startYear >= {start_year}").query(f"startYear <= {end_year}")


m = MovieWithOutputs()
m._output_df()

,primaryTitle,startYear
0,Carmencita,1894
1,Le clown et ses chiens,1892
2,Poor Pierrot,1892
3,Un bon bock,1892
4,Blacksmith Scene,1893
...,...,...
195,La fuite en Égypte,1898
196,Glasgow Fire Engine,1898
197,Gran corrida de toros,1898
198,Indian War Council,1894


Storing the outputs as a function allows for easy access to the outputs for testing, debugging and for use in other components.

On some occassions however, you might want to store the outputs as a property to cache outputs. This is useful when the output is expensive to compute.

In [16]:
class MovieWithDependentOutputs(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv(
            "./data/title.basics.tsv.gz",
            sep="\t",
            usecols=["primaryTitle", "startYear"],
            nrows=200,
        )

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = self.df["startYear"].min()
        self.end_year = self.df["startYear"].max()

    @pm.depends("df", "start_year", "end_year", watch=True)
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        self.filtered_df = df.query(f"startYear >= {start_year}").query(
            f"startYear <= {end_year}"
        )
        return self.filtered_df


m = MovieWithDependentOutputs()
m.filtered_df

,primaryTitle,startYear
0,Carmencita,1894
1,Le clown et ses chiens,1892
2,Poor Pierrot,1892
3,Un bon bock,1892
4,Blacksmith Scene,1893
...,...,...
195,La fuite en Égypte,1898
196,Glasgow Fire Engine,1898
197,Gran corrida de toros,1898
198,Indian War Council,1894


## View

Finally, we can create a view that presents the state, derived values and the outputs. I like to make this part of a `panel()` method that returns the layout. I also initialize any `pn.widgets` in this method. And by using `from_param` method, two way bindings between the parameters and the widgets are automatically created.


In [25]:
class MoviesView(pm.Parameterized):
    start_year = pm.Integer()
    end_year = pm.Integer()
    df = pm.DataFrame()
    filtered_df = pm.DataFrame()

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        basics = pd.read_csv("./data/title.basics.tsv.gz", sep="\t", nrows=500)
        ratings = pd.read_csv("./data/title.ratings.tsv.gz", sep="\t", nrows=500)
        self.df = basics.merge(ratings, on="tconst").dropna()

    @pm.depends("df", watch=True)
    def _update_bounds(self):
        self.start_year = self.df["startYear"].min()
        self.end_year = self.df["startYear"].max()

    @pm.depends("df", "start_year", "end_year", watch=True)
    def _output_df(self):
        df, start_year, end_year = (
            self.df,
            self.start_year,
            self.end_year,
        )
        self.filtered_df = df.query(f"startYear >= {start_year}").query(
            f"startYear <= {end_year}"
        )
        return self.filtered_df

    def panel(self):
        return pn.Column(
            pn.widgets.IntInput.from_param(self.param.start_year, name="Start Year"),
            pn.widgets.IntInput.from_param(self.param.end_year, name="End Year"),
            pn.widgets.Tabulator.from_param(
                self.param.filtered_df, pagination="remote", page_size=5
            ),
        )


m = MoviesView()
m.panel()

Column(sizing_mode='stretch_width')
    [0] IntInput(name='Start Year', sizing_mode='stretch_width', value=np.int64(1892))
    [1] IntInput(name='End Year', sizing_mode='stretch_width', value=np.int64(1912))
    [2] Tabulator(name='Filtered df', page_size=5, pagination='remote', sizing_mode='stretch_width', value=        tconst titleType  ...)

So when a user updates the start year or the end year, the filtered dataframe is updated. And when that filtered data is updated, the `Tabulator` widget is updated. This is a very simple example, but this pattern can be extended to more complex dashboards.

## Example: Movies Dashboard

In [17]:
#| code-fold: true

import tips_and_tricks_using_panel as tt

We can instantiate the `Movies` class.

In [132]:
m = tt.movies.Movies();

And call the `panel()` method to get an interactive dashboard.

In [143]:
m.panel()

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] RangeSlider(end=10, name='Ratings range', sizing_mode='stretch_width', step=1, value=(0, 10), value_end=10)
        [1] RangeSlider(end=2025, name='Year range', sizing_mode='stretch_width', start=1894, step=1, value=(1894, 2025), value_end=2025, value_start=1894)
        [2] RangeSlider(end=59460, name='Runtime range', sizing_mode='stretch_width', start=1, step=1, value=(1, 59460), value_end=59460, value_start=1)
    [1] Row(sizing_mode='stretch_width')
        [0] Select(name='Genre', options=OrderedDict({'Romance': 'R...]), sizing_mode='stretch_width', value='Romance')
    [2] Tabs(sizing_mode='stretch_width')
        [0] HoloViews(DynamicMap, height=400, name='Visualization', sizing_mode='fixed', width=900)
        [1] Tabulator(page_size=10, pagination='remote', sizing_mode='stretch_width', value=           tconst t...)

By having methods that return subcomponents, we can easily combine them on the fly on a case by case basis.

In [120]:
#| code-fold: true
m.genre = "Documentary"
documentary_plot = m._update_plot()
m.genre = "Comedy"
comedy_plot = m._update_plot()

m.genre = "Romance"

(documentary_plot + comedy_plot).cols(1)

:Layout
   .BoxWhisker.I  :BoxWhisker   [startYear]   (averageRating)
   .BoxWhisker.II :BoxWhisker   [startYear]   (averageRating)